# Radial Basis Function Network (RBFN) Tutorial

Laura S. Mendoza

<div style="text-align: right"> Groupe de Travail ML </div>

<div style="text-align: right"> 14 avril 2020 </div>


## Overview

- Réseau de Neurones qui utilise des **fonctions à bases radiales** comme fonctions d'activation
- Apprentissage supervisé
- Utilisations : **Approximations de fonctions** (espaces à grandes dimensions), classifications, problèmes mal posés...
- **Motivation** : utiliser les principes d'interpolation de fonctions à multiples variables dans le RN
- Première formulation par Broomhead et Lowe en 88[[1]](https://sci2s.ugr.es/keel/pdf/algorithm/articulo/1988-Broomhead-CS.pdf), puis, Powel en 87[[2]](https://ci.nii.ac.jp/naid/10000092862/), Govindaraju R.S., Zhang B. (2000)[[3]](https://link.springer.com/chapter/10.1007/978-94-015-9341-0_6), ...

## Fonction à base radiale
- Une fonction à base radiale (RBF) est une fonction $\phi$ symétrique autour d’un centre $x_i$ et avec une certaine étendue $\sigma$ : $\phi_j(x) = \phi(||x− x_j||, \sigma)$, avec $||.||$ la norme éuclidienne.
- Les RBF les plus utilisées: gaussiennes de même ampleur (1)

$$\phi(x) = \exp({-\beta||x-x_i||^2})$$

<img src="images/diff_variances_plot.png" align="center"/>


<sub><sup>[source image](http://mccormickml.com/2013/08/15/radial-basis-function-network-rbfn-tutorial/)</sup></sub>

## Principe
<img src="images/architecture_simple2.png" align="center"/>

Un RBFN est composé de 3 couches: entrée, **une** couche cachée, sortie


<sub><sup>[source image](http://mccormickml.com/2013/08/15/radial-basis-function-network-rbfn-tutorial/)</sup></sub>

## Principe

3 couches :
- **input** : vecteur de dimension $n$, tout le vecteur est donné à chacun des noeuds
- **neurones RBF** : ont chacune un *prototype* (vecteur du training set, le *centre* des RBF), retourne une valeur entre $0$ (le plus éloigné) et $1$ (le plus proche du prototype), activation.
- **output** : vecteur des *scores* $F$ pour chaque catégorie (souvent : positif si dans la catégorie, négatif sinon)

$$ F(x) = \sum_{i=0}^N \omega_i \phi_i(||x− x_j||, \sigma)$$

avec $N$ le nombre neurones et $\omega_i$ les poids associés à chaque noeud.

## Comparaison aux réseaux MLP (Multi-layer Perceptron)

- Plus rapide et facile (pas besoin de back-propagation pour la phase d'apprentissage)
- Sens clair à chaque noeud du réseaux
- Pas besoin de calculer le nombre noeuds ou de couches cachées
- Mauvaise pour les données bruitées

## Phase d'apprentissage

Il y a 4 paramètres à déterminer :

- Le nombre de neurones RBF.
- La position des centres des gaussiennes de chacun des neurones.
- La largeur de ces gaussiennes.
- Le poids des connexions entre les neurones RBF et les neurones de sortie.

Apprentissage séquentiel de chacun des paramètres (Moody, 89[[5]](https://www.scirp.org/(S(czeh2tfqyw2orz553k1w0r45))/reference/ReferencesPapers.aspx?ReferenceID=2125361))

## Phase d'apprentissage

- L’apprentissage est souvent hybride :  
    - non  supervisé  pour l’étape de construction du réseau
    - supervisé pour la détermination des poids de la couche de sortie.
- Notamment 3 schémas utilisés, décrits par Schwenker et al.[[4]](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.109.312&rep=rep1&type=pdf) : à une, deux et trois phases.

- Nombre de RBF : pas de méthode, au choix, plus de neurones => plus c'est lent et précis. Particulier à chaque problème
- Séléction prototypes (centres RBF) : aléatoire, **k-moyennes** clustering, quantification vectorielle et arbres de décision, ...
- Largeur gaussienne: plusieurs formules directes utilisées, dépendent des prototypes
- Apprentissage des poids : gradient conjugué, pseudo inversement.

$ \begin{pmatrix}
\varphi_1(||x_1 - c_1||) & ... & \varphi_M(||x_1 - c_M||) \\
\varphi_1(||x_2 - c_1||) & ... & \varphi_M(||x_2 - c_M||) \\
... & ... & ... \\
\varphi_1(||x_N - c_1||) & ... & \varphi_M(||x_N - c_M||) 
\end{pmatrix}   \begin{pmatrix}
\omega_1 \\
\omega_2 \\
... \\
\omega_N 
\end{pmatrix} = \begin{pmatrix}
y_1 \\
y_2 \\
... \\
y_N 
\end{pmatrix} $


## Exemple d'algorithme

<img src="images/cluster_centers1.png" align="center"/>

<sub><sup>[source image](http://mccormickml.com/2013/08/15/radial-basis-function-network-rbfn-tutorial/)</sup></sub>

## Algorithme

- 2 passages de **k-moyennes** (k=10) sur les 2 catégories des données d'entrainement => 20 centres de clusters
- Résultat : prototypes des RBFs (centres)
- Définir l'ampleur $\sigma$ de la RBF à la moyenne de la distance entre le centre du cluster et chaque donnée du cluster
$$ \sigma = \dfrac{1}{m}\sum_{i=1}^m || x_i - c ||  $$
$$ \beta = \dfrac{1}{2\sigma^2}$$
$$\phi(x) = \exp({-\beta||x-c||^2})$$

- On calcule les poids d'approximation $\omega_i$ par un gradient conjugué (sur les valeurs de $\phi(x)$)

<img src="images/category_1_scores_w_prototypes.png" align="center"/>

<sub><sup>[source image](http://mccormickml.com/2013/08/15/radial-basis-function-network-rbfn-tutorial/)</sup></sub>

## Exemple : XOR

In [6]:
import numpy as np
from sklearn.cluster import KMeans

# Creation data
xx, yy = np.meshgrid(np.linspace(-3, 3, 50),
                     np.linspace(-3, 3, 50))
rng = np.random.RandomState(0)
X = rng.randn(200, 2)
Y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

def rbf(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

# Calcul des centres
n_neuro = 10
kmeans = KMeans(n_clusters=n_neuro, random_state=0).fit(X)
centers = kmeans.cluster_centers_
sig = np.mean()